In [ ]:
!pip install git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-dvjx_amb
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-dvjx_amb
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install -q -U datasets accelerate bitsandbytes peft

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF_TOKEN')

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from datasets import load_dataset, DatasetDict
from peft import LoraConfig, get_peft_model

In [ ]:
model_name = "google/gemma-3-1b-pt"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, token=HF_TOKEN)

In [ ]:
dataset = load_dataset('shuttie/dadjokes')

In [ ]:
def format_joke(joke):
    return (
        f"<start_of_turn>user\n{joke['question']}<end_of_turn>\n"
        f"<start_of_turn>assistant\n{joke['response']}<end_of_turn>"
    )

formatted_dataset = dataset.map(lambda joke: {'text': format_joke(joke)})

In [ ]:
def tokenize_function(joke):
    tokens = tokenizer(
        joke['text'],
        padding='max_length',
        truncation=True,
        max_length=512,
        return_tensors='pt'
    )

    tokens['labels'] = tokens['input_ids'].clone()

    return tokens

tokenized_dataset = formatted_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/52000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

In [ ]:
small_dataset = DatasetDict({
    "train": tokenized_dataset['train'].select(range(100)),
    'test': tokenized_dataset['test'].select(range(100))
})

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=['q_proj', 'v_proj'],
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, config)

In [ ]:
training_args = TrainingArguments(
    output_dir='./fine_tuned_gemma',
    eval_strategy='epoch',
    save_strategy='epoch',
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    learning_rate=2e-5,
    num_train_epochs=2,
    logging_steps=10,
    weight_decay=0.01,
    save_total_limit=2,
    fp16=True
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset['train'],
    eval_dataset=small_dataset['test']
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 2.12 MiB is free. Process 193682 has 14.74 GiB memory in use. Of the allocated memory 14.51 GiB is allocated by PyTorch, and 99.79 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
trainer.train()

It is strongly recommended to train Gemma3 models with the `eager` attention implementation instead of `sdpa`. Use `eager` with `AutoModelForCausalLM.from_pretrained('<path-to-checkpoint>', attn_implementation='eager')`.


ValueError: The model did not return a loss from the inputs, only the following keys: logits. For reference, the inputs it received are input_ids,attention_mask.